In [1]:
# -*- coding: utf-8 -*-
"""
@量化交流QQ群:461470781，欢迎交流
"""
import pandas as pd

# ==========导入上证指数的原始数据
# 注意：这里请填写数据文件在您电脑中的路径，并注意路径中斜杠的方向
index_data = pd.read_csv('../stock_data/sh000001.csv', parse_dates=['date'])
# 保留这几个需要的字段：'date', 'high', 'low', 'close', 'change'
index_data = index_data[['date', 'high', 'low', 'close', 'change']]
# 对数据按照【date】交易日期从小到大排序
index_data.sort('date', inplace=True)

print index_data.tail(5)

# ==========计算海龟交易法则的买卖点
# 设定海龟交易法则的两个参数，当收盘价大于最近N1天的最高价时买入，当收盘价低于最近N2天的最低价时卖出
# 这两个参数可以自行调整大小，但是一般N1 > N2
N1 = 20
N2 = 10

# 通过rolling_max方法计算最近N1个交易日的最高价
index_data['最近N1个交易日的最高点'] =  pd.rolling_max(index_data['high'], N1)
# 对于上市不足N1天的数据，取上市至今的最高价
index_data['最近N1个交易日的最高点'].fillna(value=pd.expanding_max(index_data['high']), inplace=True)

# 通过相似的方法计算最近N2个交易日的最低价
index_data['最近N2个交易日的最低点'] =  pd.rolling_min(index_data['low'], N1)
index_data['最近N2个交易日的最低点'].fillna(value=pd.expanding_min(index_data['low']), inplace=True)

# 当当天的【close】> 昨天的【最近N1个交易日的最高点】时，将【收盘发出的信号】设定为1
buy_index = index_data[index_data['close'] > index_data['最近N1个交易日的最高点'].shift(1)].index
index_data.loc[buy_index, '收盘发出的信号'] = 1
# 当当天的【close】< 昨天的【最近N2个交易日的最低点】时，将【收盘发出的信号】设定为0
sell_index = index_data[index_data['close'] < index_data['最近N2个交易日的最低点'].shift(1)].index
index_data.loc[sell_index, '收盘发出的信号'] = 0

# 计算每天的仓位，当天持有上证指数时，仓位为1，当天不持有上证指数时，仓位为0
index_data['当天的仓位'] = index_data['收盘发出的信号'].shift(1)
index_data['当天的仓位'].fillna(method='ffill', inplace=True)

# 取1992年之后的数据，排出较早的数据
index_data = index_data[index_data['date'] >= pd.to_datetime('19930101')]

# 当仓位为1时，买入上证指数，当仓位为0时，空仓。计算从19920101至今的资金指数
index_data['资金指数'] = (index_data['change'] * index_data['当天的仓位'] + 1.0).cumprod()
initial_idx = index_data.iloc[0]['close'] / (1 + index_data.iloc[0]['change'])
index_data['资金指数'] *= initial_idx

# 输出数据到指定文件
index_data[['date', 'high', 'low', 'close', 'change', '最近N1个交易日的最高点',
            '最近N2个交易日的最低点', '当天的仓位', '资金指数']].to_csv('turtle.csv', index=False, encoding='gbk')


# ==========计算每年指数的收益以及海龟交易法则的收益
index_data['海龟法则每日涨跌幅'] = index_data['change'] * index_data['当天的仓位']
year_rtn = index_data.set_index('date')[['change', '海龟法则每日涨跌幅']].\
               resample('A', how=lambda x: (x+1.0).prod() - 1.0) * 100
print index_data.head()
print year_rtn

        date     high      low    close    change
4 2014-12-25  3073.35  2969.87  3072.54  0.033643
3 2014-12-26  3164.16  3064.18  3157.60  0.027686
2 2014-12-29  3223.86  3126.94  3168.02  0.003298
1 2014-12-30  3190.30  3130.35  3165.82 -0.000695
0 2014-12-31  3239.36  3157.26  3234.68  0.021752

[5 rows x 5 columns]
          date     high      low    close    change  最近N1个交易日的最高点  \
482 2013-01-04  2296.11  2256.56  2276.99  0.003466       2296.11   
481 2013-01-07  2293.32  2266.86  2285.36  0.003677       2296.11   
480 2013-01-08  2289.14  2262.98  2276.07 -0.004067       2296.11   
479 2013-01-09  2283.90  2259.05  2275.34 -0.000321       2296.11   
478 2013-01-10  2295.48  2268.65  2283.66  0.003656       2296.11   

     最近N2个交易日的最低点  收盘发出的信号  当天的仓位  资金指数  海龟法则每日涨跌幅  
482       2256.56      NaN    NaN   NaN        NaN  
481       2256.56      NaN    NaN   NaN        NaN  
480       2256.56      NaN    NaN   NaN        NaN  
479       2256.56      NaN    NaN   NaN        NaN 